### f220627.ipynb 에서 짠 코드를 토대로 zoo.csv 학습해보기
---

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np  # pd.get_dummies() -> DataFrame 반환 -> array로 type 변경하기 위함

from sklearn.model_selection import train_test_split

In [2]:
import os
print(os.getcwd())

D:\Kamie\mon_ML_torch


In [3]:
path = "../dataset/data-04-zoo.csv"

df = pd.read_csv(path, skiprows=19, header=None)
df.columns = ["hair", "feathers", "eggs", "milk", "airborne", "aquatic", "predator", "toothed", "backbone", "breathes", "venomous", "fins", "legs", "tail", "domestic", "catsize", "type"]
df.head()

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,0
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,0
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,3
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,0
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,0


In [4]:
df.shape, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   hair      101 non-null    int64
 1   feathers  101 non-null    int64
 2   eggs      101 non-null    int64
 3   milk      101 non-null    int64
 4   airborne  101 non-null    int64
 5   aquatic   101 non-null    int64
 6   predator  101 non-null    int64
 7   toothed   101 non-null    int64
 8   backbone  101 non-null    int64
 9   breathes  101 non-null    int64
 10  venomous  101 non-null    int64
 11  fins      101 non-null    int64
 12  legs      101 non-null    int64
 13  tail      101 non-null    int64
 14  domestic  101 non-null    int64
 15  catsize   101 non-null    int64
 16  type      101 non-null    int64
dtypes: int64(17)
memory usage: 13.5 KB


((101, 17), None)

In [5]:
print(f"df['type'] {df['type'].unique().size}개, {df['type'].unique()}")

df['type'] 7개, [0 3 1 6 5 4 2]


In [6]:
df['type'].value_counts()

0    41
1    20
3    13
6    10
5     8
2     5
4     4
Name: type, dtype: int64

### train, valid, test

In [7]:
x = df.drop(columns=["type"])  # (101, 16)
x = np.array(x)  # np.array로 변환
x = torch.FloatTensor(x)  # pytorch에서 사용하기 위해서 Tensor 로 변경

y = pd.get_dummies(df["type"])  # y_train을 one-hot encoding 해주기
y = np.array(y)  # np.array로 변환
y = torch.FloatTensor(y)  # pytorch에서 사용하기 위해서 Tensor로 변경
# y_train.shape : torch.Size([101, 7])

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   test_size=0.2,
                                                   stratify=y,  # target의 class 비율에 맞춰서 분리
                                                   random_state=42)

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,
                                                     test_size=0.2,
                                                     stratify=y_train,
                                                     random_state=42)

In [10]:
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

torch.Size([64, 16]) torch.Size([64, 7])
torch.Size([16, 16]) torch.Size([16, 7])
torch.Size([21, 16]) torch.Size([21, 7])


# ▶ 학습 하기 위한 준비
1. weight, bias shape 정의
2. optimizer를 뭘 사용할 것인지 등을 정의
3. for문에서 정의된 epoch 수만큼 돌면서 w, b 값 학습  
    3-1. 가설함수(hypothesis: softmax)  
    3-2. cost function: cross-entropy  

# weight 설정
```python
w = torch.zeros((), requires_grad=True)  # ()에 shape 들어가야 함
```
- x_train.shape : (101, 16)
- y_train.shape : (101, 7)
- ```x_train * w => y_train``` 나와야 함


### 101x16 matrix multiplication 16x7 => 101x7

## 방법 1 : 직접 가설함수와 cost 구현하기

In [11]:
w = torch.zeros((16, 7), requires_grad=True) # requires_grad : w 기준으로 편미분
b = torch.zeros(7, requires_grad=True)  # class가 7개 나와야 함
# bias는 wx+b로 더하는 값임. 그래서 크게 영향을 미치지는 않음

In [12]:
b.shape

torch.Size([7])

In [13]:
optimizer = optim.SGD([w, b], lr=0.1)

In [14]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 1. Hypothesis 함수 : forward
    # softmax = exp(wx+b) / sum(exp(wx+b))
    # => 구현되어 있는게 F.softmax
    hx = F.softmax(x_train.matmul(w) + b, dim=1)
    
    # 2. Cost Function 사용
    # cross-entropy = 1 / n(y*-log(h(x)))  : *는 element-wise 곱
    cost = (y_train * -torch.log(hx)).sum(dim=1).mean()  # y_one_hot는 GT값
    
    # 3. 최적화 : cost function을 미분하고, 경사타고 내려오면서 w,b 업데이터
    optimizer.zero_grad()
    cost.backward()  # cost function 미분
    optimizer.step()  # SGD, lr 만큼 내려가면서 w, b 업데이트
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f}")

Epoch    0/1000, cost: 1.945910
Epoch  100/1000, cost: 0.479482
Epoch  200/1000, cost: 0.336461
Epoch  300/1000, cost: 0.264270
Epoch  400/1000, cost: 0.217741
Epoch  500/1000, cost: 0.184771
Epoch  600/1000, cost: 0.160124
Epoch  700/1000, cost: 0.141008
Epoch  800/1000, cost: 0.125769
Epoch  900/1000, cost: 0.113353
Epoch 1000/1000, cost: 0.103058


## 방법 2 : 함수 갖다 쓰기

In [15]:
model = nn.Linear(16, 7)  # 16x7 : 입력될 특징 16가지, 출력될 값 7가지
optimizer = optim.SGD(model.parameters(), lr=0.1)
# optimizer = optim.SGD([w, b], lr=0.1) 사용하면 우리가 만든 zeros로 계속 학습하기 때문에 cost가 업데이트 안됨!

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 1. Hypothesis 함수 : forward
    # hx = F.softmax(x_train.matmul(w) + b, dim=1)
    
    # 2. Cost Function 사용
    # cost = (y_one_hot * -torch.log(hx)).sum(dim=1).mean()  # y_one_hot는 GT값
    
    # 방법1과 다른 부분
    # 1, 2 대신 아래 두 줄로 변경해도 됨. 1, 2번의 코드(hx, cost)는 dim 등 직접 구현해서 이해하기 위함
    pred = model(x_train)
    cost = F.cross_entropy(pred, y_train)
    
    # 3. 최적화 : cost function을 미분하고, 경사타고 내려오면서 w,b 업데이터
    optimizer.zero_grad()
    cost.backward()  # cost function 미분
    optimizer.step()  # SGD, lr 만큼 내려가면서 w, b 업데이트
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, cost: {cost.item():.6f}")

Epoch    0/1000, cost: 2.261313
Epoch  100/1000, cost: 0.488427
Epoch  200/1000, cost: 0.334380
Epoch  300/1000, cost: 0.260135
Epoch  400/1000, cost: 0.213504
Epoch  500/1000, cost: 0.180921
Epoch  600/1000, cost: 0.156745
Epoch  700/1000, cost: 0.138071
Epoch  800/1000, cost: 0.123218
Epoch  900/1000, cost: 0.111129
Epoch 1000/1000, cost: 0.101110


# >>>>>> 모델 평가해보기

In [16]:
model = nn.Linear(16, 7) 
optimizer = optim.SGD(model.parameters(), lr=0.1)
# optimizer = optim.SGD([w, b], lr=0.1) 사용하면 우리가 만든 zeros로 계속 학습하기 때문에 cost가 업데이트 안됨!

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    pred = model(x_train)
    train_cost = F.cross_entropy(pred, y_train)
    
    # 3. 최적화 : cost function을 미분하고, 경사타고 내려오면서 w,b 업데이트
    optimizer.zero_grad()
    train_cost.backward()  # cost function 미분
    optimizer.step()  # SGD, lr 만큼 내려가면서 w, b 업데이트
    
    model.eval()  # 학습할 때만 필요한 dropout, batchnorm 등의 기능을 비활성화
    
    with torch.no_grad():  # autograd 끄기
        val_pred = model(x_valid)
        val_cost = F.cross_entropy(val_pred, y_valid)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch:4d}/{nb_epochs}, train_cost: {train_cost.item():.6f}, val_cost: {val_cost.item():.6f}")

Epoch    0/1000, train_cost: 1.922924, val_cost: 1.698217
Epoch  100/1000, train_cost: 0.493239, val_cost: 0.475424
Epoch  200/1000, train_cost: 0.342260, val_cost: 0.314550
Epoch  300/1000, train_cost: 0.266681, val_cost: 0.239115
Epoch  400/1000, train_cost: 0.218658, val_cost: 0.195032
Epoch  500/1000, train_cost: 0.184990, val_cost: 0.166033
Epoch  600/1000, train_cost: 0.160018, val_cost: 0.145456
Epoch  700/1000, train_cost: 0.140762, val_cost: 0.130092
Epoch  800/1000, train_cost: 0.125472, val_cost: 0.118195
Epoch  900/1000, train_cost: 0.113050, val_cost: 0.108725
Epoch 1000/1000, train_cost: 0.102769, val_cost: 0.101017


# Numpy to Tensor 또는 Tensor to Numpy
```python
import numpy as np
x1 = np.ndarray(shape=(2,3), dtype=int,buffer=np.array([1,2,3,4,5,6]))
# array([[1, 2, 3],
#        [4, 5, 6]])

torch.from_numpy(x1)
# tensor([[1, 2, 3],
#         [4, 5, 6]], dtype=torch.int32)
        
x2 = torch.from_numpy(x1)
x2.numpy()
# array([[1, 2, 3],
#        [4, 5, 6]])

x2.float()
# tensor([[1., 2., 3.],
#         [4., 5., 6.]])
```

In [20]:
test_pred = model(x_test)
test_pred = F.softmax(test_pred)

test_pred_ = [test_p.argmax() for test_p in test_pred]
test_pred_

C:\Users\AI-00\AppData\Local\Temp/ipykernel_36876/4191788041.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_pred = F.softmax(test_pred)


[tensor(4),
 tensor(1),
 tensor(6),
 tensor(0),
 tensor(3),
 tensor(0),
 tensor(0),
 tensor(5),
 tensor(0),
 tensor(5),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(0),
 tensor(6),
 tensor(0),
 tensor(3),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(3)]

In [18]:
y_test_ = [y.argmax() for y in y_test]
y_test_

[tensor(4),
 tensor(1),
 tensor(6),
 tensor(0),
 tensor(3),
 tensor(0),
 tensor(0),
 tensor(5),
 tensor(0),
 tensor(5),
 tensor(1),
 tensor(0),
 tensor(2),
 tensor(0),
 tensor(6),
 tensor(0),
 tensor(3),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(3)]

In [35]:
correct = 0
for test_p, test_y in zip(test_pred_, y_test_):
    # print(test_p, test_y)
    if (test_p == test_y): 
        correct += 1
        
acc = correct / len(test_pred_) * 100.0
print(round(acc, 2))

100.0
